In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split

In [63]:
ratings_df = pd.read_csv('C:/Users/papad/Desktop/goodbooks-10k-master/goodbooks-10k-master/ratings.csv')
books_df = pd.read_csv('C:/Users/papad/Desktop/goodbooks-10k-master/goodbooks-10k-master/books.csv')
to_read_df = pd.read_csv('C:/Users/papad/Desktop/goodbooks-10k-master/goodbooks-10k-master/to_read.csv')
book_tags_df = pd.read_csv('C:/Users/papad/Desktop/goodbooks-10k-master/goodbooks-10k-master/book_tags.csv')
tags_df = pd.read_csv('C:/Users/papad/Desktop/goodbooks-10k-master/goodbooks-10k-master/tags.csv')

# Dataset

In [3]:
percentage = 1
ratings_df = ratings_df.head(int(len(ratings_df)*(percentage/100)))

In [4]:
books_df[books_df['book_id']==9439]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
9438,9439,35539,35539,1654509,17,316013315,9.780316e+12,Bob Spitz,2005.0,The Beatles: The Biography,...,8120,8464,390,205,314,1270,3020,3655,https://images.gr-assets.com/books/1327951066m...,https://images.gr-assets.com/books/1327951066s...


In [65]:
ratings_df.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [66]:
num_users = ratings_df['user_id'].unique()
print(len(num_users))
num_items = ratings_df['book_id'].unique()
print(len(num_items))

53424
10000


# Spotlight

Spotlight users and items ids should be consecutive. For that purpose in the cell below, the defaultdict function from the collections library is used to provide, the uid_map and iid_map. These variables contain consecutive numbers equal to the number of users and items respectively. These are necessary to create the interaction objects, which are the fundamental objects of the Spotlight platform, and are required in order to build a Spotlight recommender. 

Then I keep the reveresed mapped user and item ids, to take the real predicted item ids when i finish the process with the Spotlight model.

In [7]:
from collections import defaultdict
from itertools import count
uid_map = defaultdict(count().__next__)
iid_map = defaultdict(count().__next__)
uids = np.array([uid_map[uid] for uid in ratings_df["user_id"].values ], dtype=np.int32)
iids = np.array([iid_map[iid] for iid in ratings_df["book_id"].values ], dtype=np.int32)

uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}


print("userId %d got uid %d" % (556, uid_map[556]))
print("movieId %d got iid %d" % (54001, iid_map[54001]))

userId 556 got uid 1157
movieId 54001 got iid 4477


Interactions refer, to the user-item interactions. Each interaction contains a user-item pair interaction, and it can be added a timestamp, and for explicit models, a rating.

After creating the Interactions object I split the data using the random_train_test_split function from the Spotlight library. I firstly split my dataset into train set, test set, validation set. 
Having in mind that a lightGBM that combines the knowledge extracted from a few baseline models, I split the training set into features and label sets, to use them to formulate the data in order to train the lightGBM model later.

In [8]:
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split
from spotlight.factorization.implicit import ImplicitFactorizationModel
import time  


#pecify the ids of the users and items for all the pairs of user-item interactions
implicit_interaction = Interactions(user_ids=uids,
                                   item_ids=iids)


#lets initialise the seed, so that its repeatable and reproducible 
implicit_train_tmp, implicit_test = random_train_test_split(implicit_interaction, random_state=np.random.seed(42))
implicit_train, implicit_val = random_train_test_split(implicit_train_tmp, random_state=np.random.seed(42))
implicit_train_features, implicit_train_labels = random_train_test_split(implicit_train, random_state=np.random.seed(42))

In [9]:
print(implicit_train)
print(implicit_test)
print(implicit_val)
print(implicit_train_features)
print(implicit_train_labels)

<Interactions dataset (1157 users x 4477 items x 38248 interactions)>
<Interactions dataset (1157 users x 4477 items x 11953 interactions)>
<Interactions dataset (1157 users x 4477 items x 9563 interactions)>
<Interactions dataset (1157 users x 4477 items x 30598 interactions)>
<Interactions dataset (1157 users x 4477 items x 7650 interactions)>


Apart from a Spotlight model, an SVD model is trained, so the conversion of the Interaction objects to dataframes is thought as necessary, as the same data should be used for all the model, to provide fair comparisons.

In [10]:
def interactionToDataFrame(dataset):
    converted_users = []
    converted_books = []
    rating = []
    for i in range(len(dataset.user_ids)):
        converted_users.append(uid_rev_map[dataset.user_ids[i]])
        converted_books.append(iid_rev_map[dataset.item_ids[i]])
        rating.append(float(ratings_df['rating'][(ratings_df['book_id'] == converted_books[i]) & (
            ratings_df['user_id'] == converted_users[i])].values))
    df = pd.DataFrame({'user_id': converted_users,
                       'book_id': converted_books, 'rating': rating})
    return df

In [12]:
train_df = interactionToDataFrame(implicit_train)
val_df = interactionToDataFrame(implicit_val)
test_df = interactionToDataFrame(implicit_test)
train_features_df = interactionToDataFrame(implicit_train_features)
train_labels_df = interactionToDataFrame(implicit_train_labels)

## Implicit Factorization model

The model form the Spotlight, that is used is the Implicit Factorization model. This is a model used to address implicit feedback from the users. To make it clear, the implicit feedback in a movie platform could be the movie that a user watched but did not gave a rating. Given, that he/ she watched some movies, knowledge can be extracted about his/her preferences, without having ratings.

In [67]:
implicit_model = ImplicitFactorizationModel(n_iter=5)

current = time.time()

implicit_model.fit(implicit_train, verbose=True)

end = time.time()
diff = end - current
print('Training took %d second' % (diff))

Epoch 0: loss 0.8815360089143117
Epoch 1: loss 0.519373015165329
Epoch 2: loss 0.4345216010014216
Epoch 3: loss 0.3803341750303904
Epoch 4: loss 0.334525239666303
Training took 22 second


### Top - K

The idea is to provide the top-K(100) recommendations for each user and then evaluate the results using the NDCG evaluation metric. 

For this purpose on the cell below, the topK recommendations for a user are calculated.

The function below, topK_implicit, has been introduced on the lab of the course Recommender Systems.

In [14]:
import scipy.stats as st
k = 100
def topK_implicit(user, model, k=100):
  a= list()
  ranks = st.rankdata(-model.predict(user))   
  for iid in np.argwhere(ranks <= k):
    a.append(iid_rev_map[iid.item(0)])
  return a

rec_forUser = topK_implicit(0,implicit_model)
print(rec_forUser)

[258, 26, 33, 264, 18, 27, 21, 2, 23, 24, 255, 35, 287, 58, 111, 5, 8, 65, 45, 113, 476, 14, 55, 36, 13, 268, 867, 32, 28, 42, 249, 184, 495, 11, 426, 101, 344, 372, 122, 212, 9, 60, 238, 181, 119, 213, 587, 407, 10, 94, 516, 233, 115, 4, 230, 483, 2339, 304, 100, 92, 71, 1144, 557, 131, 662, 378, 323, 653, 78, 150, 114, 132, 22, 7, 162, 121, 436, 63, 2051, 672, 43, 57, 189, 160, 138, 29, 248, 118, 178, 354, 38, 335, 225, 37, 1005, 763, 430, 25, 104, 2914]


The topK recommendations for all users are needed, so the function below returns the topK recommendations for all the users, taking advantage of the function above.

In [15]:
def allUsers_topK_implicit(model):
    a = []
    for i in range(len(num_users)):
       a.append(topK_implicit(i,model))
    return a

### NDCG Score

In [16]:
allUsers_topK_imp = allUsers_topK_implicit(implicit_model)

To provide the NDCG value it is necessary to compute the relevance for each user. The relevance containes 0s and 1s, based on which of the topK recommended movies the user has actually watched. For every recommended movie that the user has actually watched, the value of 1 is assigned. 

The function below returns a list that contains a relevance list for each user.

In [17]:
def relevance_spotlight_implicit(allUsers_topK):
    relevance_implicit_full = []
    helper = []
    count = 0
    for i in range(1,len(num_users)+1):
        for j in range(len(allUsers_topK[count])):
            if allUsers_topK[count][j] in test_df['book_id'][test_df['user_id'] == i].values.tolist():
                helper.append(1)
            else:
                helper.append(0)
        relevance_implicit_full.append(helper)
        helper = []
        count += 1
    return relevance_implicit_full

The relevance_implicit_full variable contains the relevance list of each user for the implicit factorization model results.

In [18]:
relevance_implicit_full = relevance_spotlight_implicit(allUsers_topK_imp)

The dcg and ideal_dcg functions are taken form https://github.com/lezzago/LambdaMart/blob/master/lambdamart.py. They are necessary to compute the desired NDCG evaluation metric.

In [19]:
def dcg(scores):
    """
    Returns the DCG value of the list of scores.
    Parameters
    ----------
    scores : list
        Contains labels in a certain ranked order

    Returns
    -------
    DCG_val: int
        This is the value of the DCG on the given scores
    """
    return np.sum([
                    (np.power(2, scores[i]) - 1) / np.log2(i + 2)
                    for i in range(len(scores))
                ])

In [20]:
def ideal_dcg(scores):
    """
    Returns the Ideal DCG value of the list of scores.
    Parameters
    ----------
    scores : list
        Contains labels in a certain ranked order

    Returns
    -------
    Ideal_DCG_val: int
        This is the value of the Ideal DCG on the given scores
    """
    scores = [score for score in sorted(scores)[::-1]]
    return dcg(scores)

The function below, NDCG_forUser, given a relevance list, returns the NDCG value. In simpler words, it returns the NDCG value for a user.

In [21]:
def NDCG_forUser(relevance):
    return dcg(relevance) / ideal_dcg(relevance)

The function below, allUsers_ndcg, returns a list that contains the NDCG score for each user.

In [23]:
def allUsers_ndcg(relevance):
    allUsers_implicit_ndcg = []
    for i in range(len(num_users)):
        allUsers_implicit_ndcg.append(NDCG_forUser(relevance[i]))
    return allUsers_implicit_ndcg

In [24]:
allUsers_implicit_ndcg = allUsers_ndcg(relevance_implicit_full)

C:\Users\papad\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Fill the nan values with 0s.

In [25]:
allUsers_implicit_ndcg = pd.Series(allUsers_implicit_ndcg).fillna(0).tolist()

Compute the mean value of the NDCG scores.

In [26]:
np.array(allUsers_implicit_ndcg).mean()

0.15397051723121669

# SVD

On this section (Section 3), an SVD model is implemented, aiming to provide the top-K recommendations for each user and evaluate the results using the NDCG evaluation metric.

Pivot is used to have a matrix that has as rows the userIDs and as columns the movieIDs. The unrated, by a user, movies are assigned the value 0 instead of null. Pivot organizes a DataFrame by giving index and column values. It is helpful in the recommender systems approach, as it provides a dataframe in which, it can be seen directly any user-item rating.

In [27]:
df_book_features = train_df.pivot(
    index='user_id',
    columns='book_id',
    values='rating'
).fillna(0)

In [28]:
df_book_features

book_id,2,3,4,5,7,8,9,10,11,13,...,9972,9974,9978,9981,9985,9988,9990,9991,9995,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1942,0.0,5.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## SVD implementation

In [70]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(df_book_features)

## Predictions

To get the prediction of a rating of an item dj, we can calculate the dot product of the two vectors.
The sigma variable has just the values, so I have to convert to a diagonal matrix in orded to implement the matrix multiplication.

In [30]:
sigma = np.diag(sigma)
svd_predictions_score = np.dot(np.dot(U, sigma), Vt)
svd_predictions_score_df = pd.DataFrame(svd_predictions_score, columns = df_book_features.columns)


In [31]:
svd_predictions_score_df

book_id,2,3,4,5,7,8,9,10,11,13,...,9972,9974,9978,9981,9985,9988,9990,9991,9995,10000
0,-0.018121,-0.000636,-0.038918,0.019442,-0.014169,-0.009793,0.009749,0.042809,0.073432,-0.039902,...,-0.005805,-0.000236,2.671828e-03,-0.000205,0.000089,-0.002415,0.001159,0.002083,-0.000581,-0.001591
1,0.127238,-0.004687,0.128650,0.148322,-0.086465,0.286169,0.506771,-0.248870,0.471236,0.020224,...,-0.003932,0.007384,-2.558957e-03,0.014402,0.001666,0.007575,-0.005640,-0.006050,0.011011,0.001022
2,2.541552,0.062446,3.025831,2.404362,0.880952,3.261707,1.816708,0.307512,1.672557,2.315223,...,0.283308,0.045993,2.578887e-03,0.033514,0.002543,0.071791,0.007074,-0.023180,0.063451,0.047924
3,-0.001193,-0.000101,0.004147,0.002147,0.000228,0.001877,-0.000106,0.002598,0.001366,0.001033,...,-0.000039,-0.000007,-2.382167e-07,-0.000010,-0.000001,0.000031,0.000027,0.000029,-0.000011,0.000002
4,0.191701,-0.009074,0.596409,0.264347,2.166535,0.894976,0.161220,-0.927977,0.533911,1.962155,...,0.164379,0.026201,9.116976e-03,0.004001,0.001641,0.050841,-0.007299,-0.008501,0.042126,0.025034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,-0.083032,-0.022532,-0.252353,0.950737,-0.293996,0.508350,-0.160838,0.262190,2.414528,-0.343858,...,0.016384,0.031089,-1.512712e-02,-0.022181,-0.002101,-0.056608,0.053052,0.038568,-0.016694,-0.028423
1140,0.032172,0.003080,-0.038088,-0.040053,0.042527,-0.056783,-0.038017,0.026943,-0.040879,0.008380,...,0.002403,-0.000747,2.172314e-03,-0.001642,-0.000131,-0.001223,0.000581,0.000869,-0.000596,-0.000075
1141,0.211959,0.022654,-0.152217,0.285332,0.735002,0.336706,0.511541,0.400368,0.163124,0.150557,...,-0.052097,-0.005625,3.814722e-02,0.012597,0.003445,0.006011,-0.008234,0.006909,0.008350,-0.001186
1142,1.318081,0.041953,0.692827,0.350142,0.187169,0.749959,0.680729,0.026153,0.712911,0.422899,...,0.029347,0.007891,4.062414e-03,0.013104,0.000907,0.009672,-0.002507,-0.010226,0.010152,0.013842


In [32]:
svd_predictions_score_df[svd_predictions_score_df.index == 1137]

book_id,2,3,4,5,7,8,9,10,11,13,...,9972,9974,9978,9981,9985,9988,9990,9991,9995,10000
1137,2.181513,0.105843,-0.100297,0.283075,0.228997,0.434382,0.46068,0.765618,1.190321,-0.449419,...,-0.058277,0.004737,0.000695,-0.008956,-0.000542,-0.047377,0.024473,0.007537,-0.031506,-0.005032


## Top-K recommendations

The function below returns a list that contains the top-100 recommendations for each user

In [33]:
def SVD_topK_first(score_df):
    topK_svd_rec = []
    c = 0
    for i in range(len(svd_predictions_score_df)):
        svd_h_df = pd.DataFrame({'book_id': score_df[score_df.index == i].columns,
                                 'score': score_df[score_df.index == i].values[0]})
        svd_h_df = svd_h_df.sort_values('score', ascending=False)
        svd_h_df = svd_h_df.head(100)
        topK_svd_rec.append(svd_h_df['book_id'].values)
    return topK_svd_rec

## NDCG Score

The function below returns the relevance list of a list of movies.

In [34]:
def SVD_relevance_first(score_df):
    svd_topK = SVD_topK_first(score_df)
    relevance_svd_full = []
    helper = []
    count = 0
    for i in range(1,len(svd_predictions_score_df)+1):
        for j in range(len(svd_topK[count])):
            if svd_topK[count][j] in test_df['book_id'][test_df['user_id'] == i].values.tolist():
                helper.append(1)
            else:
                helper.append(0)
        relevance_svd_full.append(helper)
        helper = []
        count += 1
    return relevance_svd_full

In [35]:
relevance_first = SVD_relevance_first(svd_predictions_score_df)

In [36]:
np.asarray(relevance_first).shape

(1144, 100)

In [37]:
n_f = NDCG_forUser(relevance_first[0])

In [38]:
n_f

0.19195872000656014

Below is the NDCG scores of each user

In [39]:
def allUsers_ndcg_svd(relevance):
    allUsers_implicit_ndcg = []
    for i in range(len(svd_predictions_score_df)):
        allUsers_implicit_ndcg.append(NDCG_forUser(relevance[i]))
    return allUsers_implicit_ndcg
all_ndcg_f = allUsers_ndcg_svd(relevance_first)

C:\Users\papad\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


In [40]:
all_ndcg_f = pd.Series(all_ndcg_f).fillna(0).tolist()

In [41]:
np.asarray(all_ndcg_f).mean()

0.1468106505255648

# LightGBM Baseline

##  Feature 1 (Implicit factorization model scores)

Train an implicit factorization model using the train_features dataset

In [42]:
f1_model = ImplicitFactorizationModel(n_iter = 5)

current = time.time()

f1_model.fit(implicit_train_features,verbose = True)

end = time.time()
diff = end - current
print('Training took %d second'%(diff))

Epoch 0: loss 0.9320775464177131
Epoch 1: loss 0.5875909730792046
Epoch 2: loss 0.44491646240154903
Epoch 3: loss 0.3887164478500684
Epoch 4: loss 0.33100218152006466
Training took 25 second


### Target Movies

In [43]:
allUsers_topK_f1 = allUsers_topK_implicit(f1_model)
target_positive = allUsers_topK_f1

Add negative samples to the target movies

In [44]:
len(train_df['user_id'].unique())

1144

In [45]:
'''import random
target_negative = []
for i in range(1,len(train_features_df)+1):
    target_negative.append(random.choices(train_features_df[train_features_df['user_id']==i]['book_id'].values, k = 5))'''

"import random\ntarget_negative = []\nfor i in range(1,len(train_features_df)+1):\n    target_negative.append(random.choices(train_features_df[train_features_df['user_id']==i]['book_id'].values, k = 5))"

In [46]:
'''target_movies = []
for i in range(len(num_users)):
    target_movies.append(target_positive[i] + target_negative[i])'''
target_movies = target_positive

Map the target movies to pick the correct predictions based on the item id

In [47]:
def map_movies(target_movies):
    map_target_movies = []
    map_help = []
    for movies in target_movies:
        for movie in movies:
            map_help.append(iid_map[movie])
        map_target_movies.append(map_help)
        map_help = []
    return map_target_movies

Implicit Factorization model Scores

Train implicit on implicit_features_train! and take these scores.

In [48]:
def give_movies_implicit_scores(map_movies):
    implicit_scores = 0
    feature = []
    for i in range(len(num_users)):
        #for movies in map_target_movies:
        my_list = map_movies[i]
        my_array = np.array(my_list)
        implicit_scores = f1_model.predict(user_ids = i,item_ids = my_array)
        feature.append(implicit_scores)
    return feature

In [49]:
map_target_movies = map_movies(target_movies)
feature1 = give_movies_implicit_scores(map_target_movies)
feature1 = np.asarray(feature1)
feature1.shape

(1157, 100)

In [50]:
#feature1 = np.asarray(feature1).reshape(len(num_users),100)

## Feature 2 (Rank)

In [51]:
feature2 = []
for i in range(len(num_users)):
    my_list = map_target_movies[i]
    my_array = np.array(my_list)
    feature2.append(st.rankdata(f1_model.predict(i,my_array)))

In [52]:
np.array(feature2).shape

(1157, 100)

## Feature 3 (SVD scores)

SVD model Scores. Take advantage of the DataFrame containing the scores to extract the predicted scores for the desired movies.

In [53]:
df_movie_features1 = train_features_df.pivot(
    index='user_id',
    columns='book_id',
    values='rating'
).fillna(0)

In [54]:
U1, sigma1, Vt1 = svds(df_movie_features1)
sigma1 = np.diag(sigma1)
svd_predictions_score1 = np.dot(np.dot(U1, sigma1), Vt1)
svd_predictions_score_df1 = pd.DataFrame(svd_predictions_score1, columns = df_movie_features1.columns)

In [55]:
target_movies[0][2]

27

In [56]:
svd_predictions_score_df1[svd_predictions_score_df1.index == 0]

book_id,2,3,4,5,7,8,9,10,11,13,...,9967,9972,9974,9978,9985,9988,9990,9991,9995,10000
0,-0.023187,-0.001517,0.042136,0.03971,-0.008424,0.033313,-0.000785,0.008561,0.035325,0.01221,...,-0.000281,0.000299,0.000086,-0.000059,-0.000019,0.000227,0.000166,0.000818,0.000438,0.000015


In [57]:
float(svd_predictions_score_df1[svd_predictions_score_df1.index == 0][target_movies[0][0]].values)

0.001783205801175895

In [58]:

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(svd_predictions_score_df1[svd_predictions_score_df1.index == 0])

book_id     2         3         4        5         7         8         9      \
0       -0.023187 -0.001517  0.042136  0.03971 -0.008424  0.033313 -0.000785   

book_id     10        11       13        14        15        18        19     \
0        0.008561  0.035325  0.01221  0.014896  0.002697 -0.027124 -0.012657   

book_id    21        22        23        24        25        26        27     \
0       -0.02927  0.016642 -0.018084 -0.027794 -0.013751  0.009639 -0.027855   

book_id     28        29        32        33        35        36        37     \
0        0.007657  0.002535  0.012593  0.015713  0.017865  0.005079 -0.001239   

book_id     38        39        40        41        42        43        44     \
0        0.017746 -0.002295  0.004163  0.000472  0.005093  0.009849  0.002084   

book_id     45        46        47        48        49        50        51     \
0        0.017375  0.006553  0.001168 -0.000048 -0.000785  0.004511 -0.000029   

book_id     52        53    

In [59]:
feature3 = []
feature3_user = []
count = 0
for i in range(len(num_users)):
    for j in range(len(target_movies[i])):
        feature3_user.append(float(svd_predictions_score_df1[svd_predictions_score_df1.index == i][target_movies[i][j]].values))
    feature3.append(feature3_user)
    feature3_user = []

KeyError: 2622

In [ ]:
type(feature3)

## Feature 4 (Genre)

In [ ]:
def create_genre_feature(genre):
    feature = []
    genre_per_user = []
    for i in range(len(num_users)):
        for j in range(len(target_movies[i])):
            a =  movies_df[movies_df['movieId'] == target_movies[i][j]]['genres']
            a = a.tolist()
            a = str(a[0])
            a = a.split('|')
            if genre in a:
                genre_per_user.append(1)
            else:
                genre_per_user.append(0)
        feature.append(genre_per_user)
        genre_per_user = []
    return feature

In [ ]:
feature4 = create_genre_feature('Comedy')

### Features 8-15 (genres)

In [ ]:
feature8 = create_genre_feature('Adventure')
feature9 = create_genre_feature('Drama')
feature10 = create_genre_feature('Romance')
feature11 = create_genre_feature('Fantasy')
feature12 = create_genre_feature('Thriller')
feature13 = create_genre_feature('Animation')
feature14 = create_genre_feature('Children')
feature15 = create_genre_feature('Horror')

## Feature 5 (#Users watched each target movie)

In [ ]:
number_views = []
number_view_per_movie = []
for i in range(len(num_users)):
    for j in range(len(target_movies[i])):
        a = len(ratings_df[ratings_df['book_id'] == target_movies[i][j]]['user_id'])
        number_view_per_movie.append(a)
    number_views.append(number_view_per_movie)
    number_view_per_movie = []


In [ ]:
feature5 = number_views

In [ ]:
np.array(feature5).shape

## Feature 6 (New user)

In [ ]:
len(ratings_df[ratings_df['user_id']==1]) > 10

In [ ]:
new_users_binary = []
binary_per_user = []
for i in range(len(num_users)):
    for j in range(len(target_movies[i])):
        a = len(ratings_df[ratings_df['user_id']==1]) > 10
        if a:
            binary_per_user.append(1)
        else:
            binary_per_user.append(0)
    new_users_binary.append(binary_per_user)
    binary_per_user = []

In [ ]:
#rankings of old/new users

In [ ]:
feature6 = new_users_binary

In [ ]:
np.array(feature6).shape

## Feature 7(Embeddings) x32

In [ ]:
emb = f1_model._net.item_embeddings

In [ ]:
emb_features = []
helper = []
for i in range(32):
    for j in range(len(num_items)):
        helper.append(emb.weight[j][i].item())
    emb_features.append(helper)
    helper = []

In [ ]:
a = []
b = []
feature7 = []
for i in range(32):
    for user in range(len(num_users)):
        for movie in map_target_movies[user]:
            a.append(emb_features[0][movie])
        b.append(a)
        a = []
    feature7.append(b)
    b = []

In [ ]:
np.array(feature7).shape

In [ ]:
np.array(feature7[0]).shape

## Features stack

In [ ]:
'''features = np.stack((feature1,feature2,feature3,feature4,feature5,feature6,
                     feature7[0], feature7[1],feature7[2], feature7[3],
                     feature7[4], feature7[5],feature7[6], feature7[7],
                     feature7[8], feature7[9],feature7[10], feature7[11],
                     feature7[12], feature7[13],feature7[14], feature7[15],
                     feature7[16], feature7[17],feature7[18], feature7[19],
                     feature7[20], feature7[21],feature7[22], feature7[23],
                     feature7[24], feature7[25],feature7[26], feature7[27],
                     feature7[28], feature7[29],feature7[30], feature7[31],
                     feature8,feature9,feature10,feature11,feature12,
                    feature13,feature14,feature15))'''
features = np.stack((feature1,feature2,feature5,feature6,
                     feature7[0], feature7[1],feature7[2], feature7[3],
                     feature7[4], feature7[5],feature7[6], feature7[7],
                     feature7[8], feature7[9],feature7[10], feature7[11],
                     feature7[12], feature7[13],feature7[14], feature7[15],
                     feature7[16], feature7[17],feature7[18], feature7[19],
                     feature7[20], feature7[21],feature7[22], feature7[23],
                     feature7[24], feature7[25],feature7[26], feature7[27],
                     feature7[28], feature7[29],feature7[30], feature7[31]))

In [ ]:
len(features)

## Labels

Below are the movies that each user watched

In [ ]:
def watched_movies(df):
    watched = []
    for i in range(len(num_users)):
        watched.append(df[df["user_id"]==uid_rev_map.get(i)].groupby("book_id")["book_id"].count().index)  
    return watched

In [ ]:
#train_labels_df
#test_df
#val_df
train_watched = watched_movies(train_labels_df)
test_watched = watched_movies(test_df)
val_watched = watched_movies(val_df)

Creation of the target list. It containes 0s and 1s for the 100 recommended movies for each user. Based on the watched movies it assigns the values. If the user watched the recommended movie, then the value of the target is equal to 1, otherwise it is 0.

In [ ]:
def create_target(watched):
    user_target = []
    target = []
    for i in range(len(num_users)):
        for target_movie in target_movies[i]:
            if target_movie in watched[i]:
                user_target.append(1)
            else:
                user_target.append(0)
        target.append(user_target)
        user_target = []
    return target

In [ ]:
label_train = create_target(train_watched)
label_test = create_target(test_watched)
label_val = create_target(val_watched)
#y_train = y_test = y_val

## LightGBM model

In [ ]:
X = np.array(features)
label_train = np.array(label_train)
label_test = np.array(label_test)
label_val = np.array(label_val)

In [ ]:
k = len(target_movies[0])
X = X.reshape(len(num_users)*k, len(features))
y_train = label_train.reshape(len(num_users)*k,)
y_test = label_test.reshape(len(num_users)*k,)
y_val = label_val.reshape(len(num_users)*k,)

In [ ]:
import lightgbm as lgb
gbm = lgb.LGBMRanker(min_child_samples=1, learning_rate=.1, n_jobs=2, importance_type='gain',
                     num_leaves=31, min_child_weight=100)

target movies
train_df
feature1
feature2

In [ ]:
'''X_train_feature1 = []
X_train_feature2 = []
for j in range(610):
    for i in range(len(target_movies[j])):
        if target_movies[j][i] in val_df['movie_id'].values.tolist():
            X_train_feature1.append(feature1[i])
            X_train_feature2.append(feature2[i])
print(X_train_feature1)
print(X_train_feature2)
#X_train = np.stack(X_train_feature1,X_train_feature2)'''

X_train
X_test
X_val
y_train
y_test
y_val

In [ ]:
X_train = X
X_test = X
X_val = X

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
query_train = [X_train.shape[0]]
query_val = [X_val.shape[0]]
query_test = [X_test.shape[0]]
group_train = []
group_val = []
print(query_train)
print(query_val)
print(query_test)
'''
for i in range(390):
    group_train.append(100)
group_train.append(40)
group_val.append(60)
for i in range(97):
    group_val.append(100)'''

for i in range(len(num_users)):
    group_train.append(k)
    group_val.append(k)


In [ ]:
gbm.fit(X_train, y_train, group=group_train,
        eval_set=[(X_val, y_val)], eval_group=[group_val],
        eval_at=[1, 3, 5, 10], early_stopping_rounds=50, eval_metric='ndcg')
#no early stopping

In [ ]:
test_pred = gbm.predict(X_test)
test_pred

## NDCG Score

In [ ]:
p = 0
m = 100
lgbm_pred_per_user = []
for i in range(len(num_users)):
    lgbm_pred_per_user.append(test_pred[p:m])
    p += 100
    m += 100

### NDCG IFM on this data

In [ ]:
np.array(feature7[0][609]).shape

In [ ]:
feature1.shape

In [ ]:
lgbm_NDCG = []
IFM_NDCG = []
#SVD_NDCG = []
for i in range(len(num_users)):
    excel_df = pd.DataFrame({'movie_id': target_movies[i], 'model1_score': feature1[i],
                             'rank': feature2[i], '#users_watched': feature5[i],
                             'new_user': feature6[i],
                             'emedding1': feature7[0][i], 'emedding2': feature7[1][i], 'emedding3': feature7[2][i],
                             'emedding4': feature7[3][i], 'emedding5': feature7[4][i], 'emedding6': feature7[5][i],
                             'emedding7': feature7[6][i], 'emedding8': feature7[7][i], 'emedding9': feature7[8][i],
                             'emedding10': feature7[9][i], 'emedding11': feature7[10][i], 'emedding12': feature7[11][i],
                             'emedding13': feature7[12][i], 'emedding14': feature7[13][i], 'emedding15': feature7[14][i],
                             'emedding16': feature7[15][i], 'emedding17': feature7[16][i], 'emedding18': feature7[17][i],
                             'emedding19': feature7[18][i], 'emedding20': feature7[19][i], 'emedding21': feature7[20][i],
                             'emedding22': feature7[21][i], 'emedding23': feature7[22][i], 'emedding24': feature7[23][i],
                             'emedding25': feature7[24][i], 'emedding26': feature7[25][i], 'emedding27': feature7[26][i],
                             'emedding28': feature7[27][i], 'emedding29': feature7[28][i], 'emedding30': feature7[29][i],
                             'emedding31': feature7[30][i], 'emedding32': feature7[31][i],
                             'train_label': label_train[i], 'test_label': label_test[i],
                             'y_val': label_val[i], 'LGBM Score': lgbm_pred_per_user[i]})
    excel_df = excel_df.sort_values('LGBM Score', ascending=False)
    relevance_lgbm_user = excel_df['test_label'].values.tolist()
    lgbm_NDCG.append(NDCG_forUser(relevance_lgbm_user))
    excel_df = excel_df.sort_values('model1_score', ascending=False)
    ifm_relevance = excel_df['test_label'].values.tolist()
    IFM_NDCG.append(NDCG_forUser(ifm_relevance))
    #excel_df = excel_df.sort_values('model2_score', ascending=False)
    #svd_relevance_h = excel_df['test_label'].values.tolist()
    #SVD_NDCG.append(NDCG_forUser(svd_relevance_h))

In [ ]:
lgbm_NDCG = pd.Series(lgbm_NDCG).fillna(0).tolist()
IFM_NDCG = pd.Series(IFM_NDCG).fillna(0).tolist()
#SVD_NDCG = pd.Series(SVD_NDCG).fillna(0).tolist()

## Excel-Like

In [ ]:
excel1_df = pd.DataFrame({'movie_id': target_movies[0],'model1_score':feature1[0], 'model2_score':feature3[0],'is_comedy':feature4[0],
                            '#users_watched':feature5[0], 'train_label':label_train[0], 'test_label':label_test[0],
                            'label_val':label_val[0],'LGBM Score':lgbm_pred_per_user[0]})

In [ ]:
excel1_df.head(10)

In [ ]:
excel1_df = excel1_df.sort_values('LGBM Score',ascending = False)
excel1_df.head(10)

In [ ]:
relevance_lgbm_firstUser = excel1_df['test_label'].values.tolist()

In [ ]:
L1GBM_ndcg = NDCG_forUser(relevance_lgbm_firstUser)
L1GBM_ndcg

## NDCG Comparison

In [ ]:
allUsers_implicit_ndcg[0]

In [ ]:
ndcg_df = pd.DataFrame({'Implicit FM': allUsers_implicit_ndcg, 'LightGBM': lgbm_NDCG, 'IFM_this': IFM_NDCG})

In [ ]:
ndcg_df

In [ ]:
ndcg_df['Implicit FM'].mean()

In [ ]:
lgbm_mean = ndcg_df['LightGBM'].mean()
lgbm_mean

In [ ]:
ifm_mean = ndcg_df['IFM_this'].mean()
ifm_mean

In [ ]:
#svd_mean = ndcg_df['SVD_this'].mean()
#svd_mean

## Visualization